## Proceso 4: Clasificación de sentimientos múltiples, como: alegría, ira, culpa, etc.

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer

import sys
sys.path.append(sys.path[0] + '/..')

from comun import archivo, cadena
from preprocesamiento import tokenizacion
from aprendizajeAutomatico import entrenamiento, prediccion
from sentimociones.sentimientos.sentimientoMultiple import ALEGRIA, CULPA, DISGUSTO, IRA, MIEDO, TRISTEZA, VERGUENZA

## Lee dataset con texto en español clasificado con sentimientos múltiples

In [2]:
dataset_sentimientos_es = "sentimientoMultiple_es.txt"
data = archivo.leer_datos(dataset_sentimientos_es)
print("Number of instances: {}".format(len(data)))

Number of instances: 7480


### Función para almacenar las etiquetas, que se basarán en emociones como Alegría, Miedo, Ira, etc.:

In [3]:
emotions = [ALEGRIA, MIEDO, IRA, TRISTEZA, DISGUSTO, VERGUENZA, CULPA]

X_all = []
y_all = []
for label, text in data:
    y_all.append(cadena.convertirEtiqueta(label, emotions))
    X_all.append(tokenizacion.crear_caracteristica(text, nrange=(1, 4)))

### Separación de datos en conjuntos de entrenamiento y prueba

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = 0.2, random_state = 100)
vectorizer = DictVectorizer(sparse = True)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

### Entrenamiento con cuatro modelos de aprendizaje automático para la clasificación múltiple de sentimientos:
1. Clasificación con vectores de Soporte 
2. Clasificación con vectores de soporte lineal
3. Clasificación con randon forest
4. Clasificación con árboles de decisión

In [5]:
svc = SVC()
lsvc = LinearSVC(random_state=100)
rforest = RandomForestClassifier(random_state=100)
dtree = DecisionTreeClassifier()

clifs = [svc, lsvc, rforest, dtree]

modelos = {}
for clf in clifs: 
    modelos[clf.__class__.__name__] = entrenamiento.entrenar_probar_accu_score(clf, X_train, X_test, y_train, y_test, False)

c:\tools\ides\anaconda\envs\entorno_prueba\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


## Resultado de los modelos

In [6]:
for modelo in modelos:
    print(modelo, modelos[modelo][1])

SVC {'Precisión entrenamiento': 0.911096256684492, 'Precisión pruebas': 0.4772727272727273}
LinearSVC {'Precisión entrenamiento': 0.9991644385026738, 'Precisión pruebas': 0.5588235294117647}
RandomForestClassifier {'Precisión entrenamiento': 0.9991644385026738, 'Precisión pruebas': 0.5294117647058824}
DecisionTreeClassifier {'Precisión entrenamiento': 0.9991644385026738, 'Precisión pruebas': 0.4298128342245989}


## Definición de la etiqueta múltiple de sentimientos con su respectivo emoji

In [7]:
l = [ALEGRIA, MIEDO, IRA, TRISTEZA, DISGUSTO, VERGUENZA, CULPA]
l.sort()
label_freq = {}
for label, _ in data: 
    label_freq[label] = label_freq.get(label, 0) + 1

# imprime las etiquetas ordenadas
for l in sorted(label_freq, key=label_freq.get, reverse=True):
    print("{:10}({})  {}".format(cadena.convertirEtiqueta(l, emotions), l, label_freq[l]))

emoji_dict = {ALEGRIA:"😂", MIEDO:"😱", IRA:"😠", TRISTEZA:"😢", DISGUSTO:"😒", VERGUENZA:"😳", CULPA:"😳"}

alegría   (1. 0. 0. 0. 0. 0. 0.)  1084
ira       (0. 0. 1. 0. 0. 0. 0.)  1080
tristeza  (0. 0. 0. 1. 0. 0. 0.)  1079
miedo     (0. 1. 0. 0. 0. 0. 0.)  1078
disgusto  (0. 0. 0. 0. 1. 0. 0.)  1057
culpa     (0. 0. 0. 0. 0. 0. 1.)  1057
vergüenza (0. 0. 0. 0. 0. 1. 0.)  1045


## Obtiene letra de canción preprocesada

In [27]:
from bdd import mongoDB_cliente

cantante = 'Adele'
cancion = 'Easy On Me'

canciones = mongoDB_cliente.obtenerColeccion('CANCIONES', {'cantante': cantante, 'cancion': cancion})   

letra_preprocesada = canciones[0]['letra_preprocesada']
letra_cancion_pre = ''
for letra in letra_preprocesada:
    letra_cancion_pre = letra_cancion_pre + letra + " "
print (letra_cancion_pre)

LETRA EASY LYRICS INGLES AINT GOLD RIVER WASHIN HANDS FOREVER KNOW HOPE WATERS CANT BRING SWIM DROWNIN SILENCE BABY LET GO EASY BABY STILL CHILD DIDNT GET CHANCE FEEL WORLD AROUND TIME CHOOSE CHOSE GO EASY AINT ROOM THINGS CHANGE DEEPLY STUCK WAYS CANT DENY HARD IVE TRIED CHANGED PUT FIRST GIVE GO EASY MЕ BABY STILL CHILD DIDNT GET CHANCE FEEL THЕ WORLD AROUND TIME CHOOSE CHOSE GO EASY GOOD INTENTIONS HIGHEST HOPES KNOW RIGHT PROBABLY DOESNT EVEN SHOW GO EASY BABY STILL CHILD DIDNT GET CHANCE FEEL WORLD AROUND TIME CHOOSE CHOSE GO EASY ADELE EASY LYRICS TRANSLATE ADELE TRADUCCIÓN ORO RÍO LAVANDO MANOS SIEMPRE SÉ ESPERANZA AGUAS PUEDO NADAR AHOGO SILENCIO BEBÉ DÉJAME ENTRAR CALMA BEBÉ TODAVÍA NIÑO OPORTUNIDAD SENTIR MUNDO ALREDEDOR TIEMPO ELEGIR ELEGÍ HACER ASÍ IR FÁCIL ESPACIO COSAS CAMBIO AMBOS TAN PROFUNDAMENTE ATRAPADOS CAMINOS PUEDES NEGAR INTENTADO CAMBIÉ PONERLOS DOS PRIMERO AHORA RINDO TEN CUIDADO CONMIGO NENA TODAVÍA NIÑO ENTENDÍ OPORTUNIDAD WORLD ALREDEDOR TIEMPO ELEGIR ELEGÍ 

## Predicción del modelo con vectores de soporte

In [28]:
modelo = svc
res_prediccion, modelos = prediccion.predecirTexto(modelo, letra_cancion_pre, vectorizer, modelos, cancion, cantante)
print('Clasificación de sentimientos con vectores de soporte: ', emoji_dict[res_prediccion], res_prediccion)

Clasificación de sentimientos con vectores de soporte:  😠 ira


## Ejecución del modelo vectores de soporte lineal

In [29]:
modelo = lsvc
res_prediccion, modelos = prediccion.predecirTexto(modelo, letra_cancion_pre, vectorizer, modelos, cancion, cantante)
print('Clasificación de sentimientos con vectores de soporte: ', emoji_dict[res_prediccion], res_prediccion)

Clasificación de sentimientos con vectores de soporte:  😳 culpa


## Ejecución del modelo randon forest

In [30]:
modelo = rforest
res_prediccion, modelos = prediccion.predecirTexto(modelo, letra_cancion_pre, vectorizer, modelos, cancion, cantante)
print('Clasificación de sentimientos con vectores de soporte: ', emoji_dict[res_prediccion], res_prediccion)

Clasificación de sentimientos con vectores de soporte:  😠 ira


## Ejecución del modelo con árboles de decisión

In [21]:
modelo = dtree
res_prediccion, modelos = prediccion.predecirTexto(modelo, letra_cancion_pre, vectorizer, modelos, cancion, cantante)
print('Clasificación de sentimientos con vectores de soporte: ', emoji_dict[res_prediccion], res_prediccion)

Clasificación de sentimientos con vectores de soporte:  😂 alegría


## Actualización del resultado en Base de datos

In [31]:
from logica import cancion_logica
from sentimientos import sentimientoMultiple

cancion_logica.actualizarCancionSentimientoMultiple(canciones[0], sentimientoMultiple.obtenerResultadoPredicciones(modelos))

'documento actualizado correctamente'

## Procesamiento de todas las letras

In [25]:
from bdd import mongoDB_cliente
from logica import cancion_logica
from sentimientos import sentimientoMultiple

canciones = mongoDB_cliente.obtenerColeccion('CANCIONES', {'estado': 'LETRA PREPROCESADO'})

clifs = [svc, lsvc, rforest, dtree]
cont = 0
for c in canciones:
    cont += 1
    letra_preprocesada = c['letra_preprocesada']
    cantante = c['cantante']
    cancion = c['cancion']
    letra_cancion_pre = ''
    # organiza las letras vectorizadas en una sola cadena
    for letra in letra_preprocesada:
        letra_cancion_pre = letra_cancion_pre + letra + " "
    # ejecuta todos los modelos
    for modelo in clifs:
        res_prediccion, modelos = prediccion.predecirTexto(modelo, letra_cancion_pre, vectorizer, modelos, cancion, cantante)
        print('Clasificación de sentimientos con: ', modelo.__class__.__name__, ': ', emoji_dict[res_prediccion], res_prediccion)
    cancion_logica.actualizarCancionSentimientoMultiple(c, sentimientoMultiple.obtenerResultadoPredicciones(modelos))

print('Total de letras clasificadas', cont)

Clasificación de sentimientos con:  SVC :  😠 ira
Clasificación de sentimientos con:  LinearSVC :  😒 disgusto
Clasificación de sentimientos con:  RandomForestClassifier :  😒 disgusto
Clasificación de sentimientos con:  DecisionTreeClassifier :  😳 vergüenza
Clasificación de sentimientos con:  SVC :  😠 ira
Clasificación de sentimientos con:  LinearSVC :  😳 culpa
Clasificación de sentimientos con:  RandomForestClassifier :  😳 vergüenza
Clasificación de sentimientos con:  DecisionTreeClassifier :  😳 vergüenza
Clasificación de sentimientos con:  SVC :  😠 ira
Clasificación de sentimientos con:  LinearSVC :  😒 disgusto
Clasificación de sentimientos con:  RandomForestClassifier :  😳 culpa
Clasificación de sentimientos con:  DecisionTreeClassifier :  😂 alegría
Clasificación de sentimientos con:  SVC :  😠 ira
Clasificación de sentimientos con:  LinearSVC :  😳 vergüenza
Clasificación de sentimientos con:  RandomForestClassifier :  😒 disgusto
Clasificación de sentimientos con:  DecisionTreeClassifi